In [ ]:
import numpy as np
import tensorflow as tf
import requests
import json

# get your access token from like https://developer.spotify.com/console/get-audio-features-several-tracks/
# accessToken = ""

# ids of the track to use, maybe load from file and not from a list here? TODO
ids = ["7ouMYWpwJ422jRcDASZB7P","4VqPOruhp5EdPBeR92t6lQ","2takcwOaAZWiXQijPHIx7B"]

def nice(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))

Reference https://developer.spotify.com/documentation/web-api/reference/

TODO:
* Load a fuckton more song IDs
* Maybe do that without getting fucked by spotify
* Better model for evaluating songs

# Load dataset
Spotify doesn't send all the audio data together: it sends first a list with links, so we need to get each song analysis after that. This will sure be rate limited.

In [ ]:
header = {"Authorization" : f"Bearer {accessToken}",
                         "Accept": "application/json",
                         "Content-Type": "application/json"}

r = requests.get(f"https://api.spotify.com/v1/audio-features/?ids=7ouMYWpwJ422jRcDASZB7P%2C4VqPOruhp5EdPBeR92t6lQ%2C2takcwOaAZWiXQijPHIx7B",
                headers=header)
data = r.json()

In [ ]:
features = []
for e in data["audio_features"]:
    analysis = requests.get(e["analysis_url"], headers=header).json()
    e["analysis"] = analysis
    features.append(e)

Ok cool, but what the fuck is this "audio analysis" data?

In [ ]:
sample = features[0]["analysis"]
for key in sample:
    print(key)
print()
# What's the difference between bars and beats and tatums?
# nice(sample["segments"])
print(len(sample["segments"]))
nice(sample["track"])
nice(sample["segments"][0])

The segments list contains quite some info, and there's just too much. What to use? Should probably keep into consideration the confidence.

We can do a timeserie of shape (2, 12) with pitches and timbres, so the input array could be of shape (len(segments), 2, 12), and including batches (len(features), len(segments), 2, 12)

Issue : len(segments) is variable! Shall we just make all features as long as the longest and fill empty samples with zeroes? We can probably make the RNN stop once the timeserie ends

In [ ]:
# find the longest segments serie we have, this is sure doable with a fancy oneliner!
max_len = 0
for el in features:
    max_len = max(max_len, len(el["analysis"]["segments"]))

In [ ]:
FEATURES = np.array((len(features), max_len, 2, 12))

empty = [ [0,0,0,0,0,0,0,0,0,0,0,0], [0,0,0,0,0,0,0,0,0,0,0,0] ]

batches = []
for el in features: # this is also kinda ugly omg
    buf = []
    for seg in el["analysis"]["segments"]:
        buf.append([ seg["pitches"], seg["timbre"] ])
    for i in range(len(buf), max_len):
        buf.append(empty)
    batches.append(buf)
    print(len(batches[-1]))

FEATURES = np.array(batches)

1267
1267
1267


In [ ]:
FEATURES.shape # now the shape should be proper

(3, 1267, 2, 12)

# How to make labels
This is quite a spicy topic. We can maybe rank a n-dim space by comparing distance from the position it gives to tracks and distance in the way-less-than-n-dim space that spotify gives. For now we can just use the vector spotify gives us as labels

In [ ]:
sample = features[0]
for key in sample:
    print(key)

# We can use [danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo]
#       shape of sample label : (11)

In [ ]:
LABELS = np.empty((len(features), 11)) # len(labes) : 11

for i in range(len(features)):
    # lmao my numpy game is so rusty
    LABELS[i][0] = features[i]["danceability"]
    LABELS[i][1] = features[i]["energy"]
    LABELS[i][2] = features[i]["key"]
    LABELS[i][3] = features[i]["loudness"]
    LABELS[i][4] = features[i]["mode"]
    LABELS[i][5] = features[i]["speechiness"]
    LABELS[i][6] = features[i]["acousticness"]
    LABELS[i][7] = features[i]["instrumentalness"]
    LABELS[i][8] = features[i]["liveness"]
    LABELS[i][9] = features[i]["valence"]
    LABELS[i][0] = features[i]["tempo"]

In [ ]:
LABELS.shape

(3, 11)

# Estimating model size and params
To see if this is feasible and with what model/hyperparams, let's first make a model that classifies the songs same way spotify did, using their properties directly as labels

Model talk: we should definitely not just flatten the input but have a RNN use the timeserie property, so we "get rid of first dim". 2 other dims can be merged into one before with a Conv2D layer probably. Then some layers of LSTM / GRU. Output for now will be a Dense with 11 neurons but just to make it regenerate the spotify labels while messing with the model.

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.Input(shape=(len(FEATURES[0]), 2, 12)),
                             tf.keras.layers.Flatten(), # Sure, let's just flatten it and lose so much useful info
                             tf.keras.layers.Dense(100),
                             tf.keras.layers.Dense(50),
                             tf.keras.layers.Dense(11)
])

model.compile(loss="mse", metrics=["accuracy"], optimizer="adam")

In [ ]:
hst = model.fit(FEATURES, LABELS, epochs=10, batch_size=1) # lmao 3 samples